# Replacement Score Test

## Load the model from the local checkpoint (save from HF before)

In [1]:
import pathlib

from crosslayer_transcoder.utils.model_converters.circuit_tracer import (
    CircuitTracerConverter,
)
from transformer_lens.loading_from_pretrained import get_pretrained_model_config
import torch

In [2]:
from crosslayer_transcoder.utils.module_builder import build_module_from_config, yaml_to_config
from crosslayer_transcoder.utils.checkpoints import load_model_from_lightning_checkpoint

config = yaml_to_config("../../config/circuit-tracer.yaml")
clt_module = build_module_from_config(config)

# load checkpoint
checkpoint = "../checkpoints/clt.ckpt"

clt_module = load_model_from_lightning_checkpoint(clt_module, checkpoint)


save_dir = pathlib.Path("clt_module_test")
feature_input_hook = "hook_resid_mid"
feature_output_hook = "hook_mlp_out"

converter = CircuitTracerConverter(
save_dir=save_dir,
feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
)
converter.convert_and_save(clt_module, dtype=torch.bfloat16) 

Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1174 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1561 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2027 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2459 > 1024). Running this sequence through the model will result in indexing errors
Converting CLT : 100%|██████████| 12/12 [00:24<00:00,  2.01s/it]


In [3]:
from circuit_tracer.transcoder.cross_layer_transcoder import load_clt
transcoder, state_dict_pre_load = load_clt(
    clt_path=save_dir.as_posix(),
    lazy_decoder=False,
    lazy_encoder=False,
    feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
    dtype=torch.bfloat16,
)


12
['W_enc_0', 'b_dec_0', 'b_enc_0', 'threshold_0']
['W_enc_1', 'b_dec_1', 'b_enc_1', 'threshold_1']
['W_enc_2', 'b_dec_2', 'b_enc_2', 'threshold_2']
['W_enc_3', 'b_dec_3', 'b_enc_3', 'threshold_3']
['W_enc_4', 'b_dec_4', 'b_enc_4', 'threshold_4']
['W_enc_5', 'b_dec_5', 'b_enc_5', 'threshold_5']
['W_enc_6', 'b_dec_6', 'b_enc_6', 'threshold_6']
['W_enc_7', 'b_dec_7', 'b_enc_7', 'threshold_7']
['W_enc_8', 'b_dec_8', 'b_enc_8', 'threshold_8']
['W_enc_9', 'b_dec_9', 'b_enc_9', 'threshold_9']
['W_enc_10', 'b_dec_10', 'b_enc_10', 'threshold_10']
['W_enc_11', 'b_dec_11', 'b_enc_11', 'threshold_11']
dict_keys(['b_dec', 'b_enc', 'activation_function.threshold', 'W_enc', 'W_dec.0', 'W_dec.1', 'W_dec.2', 'W_dec.3', 'W_dec.4', 'W_dec.5', 'W_dec.6', 'W_dec.7', 'W_dec.8', 'W_dec.9', 'W_dec.10', 'W_dec.11'])


In [4]:
from circuit_tracer import ReplacementModel


config = get_pretrained_model_config("gpt2")
config.d_type = torch.float16
rm = ReplacementModel.from_config(
    config=config,
    transcoders=transcoder,
)





In [5]:
prompt = (
    "The capital of state containing Dallas is"  # What you want to get the graph for
)
max_n_logits = 10  # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 100  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size = 256//8  # Batch size when attributing
offload = (
    "cpu"
)  # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report


In [6]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils import create_graph_files

torch.cuda.empty_cache()

# FOr some reason this takes up a lot of VRAM
graph = attribute(
    prompt=prompt,
    model=rm,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose
)

Phase 0: Precomputing activations and vectors


Tokens shape: torch.Size([8])
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
l0: 70000
sparse_layer nnz: 70000
Features shape: torch.Size([12, 8, 10000])
Encoder vectors shape: torch.Size([840000, 768])
nnz: 840000


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.62 GiB. GPU 0 has a total capacity of 44.34 GiB of which 10.55 GiB is free. Process 3607992 has 8.54 GiB memory in use. Process 3613770 has 25.24 GiB memory in use. Of the allocated memory 23.21 GiB is allocated by PyTorch, and 1.72 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
graph_dir = 'graphs'
graph_name = 'example_graph.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

graph.to_pt(graph_path)